In [1]:
from pulp import *
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Set the random seed for reproducibility
np.random.seed(42)

# Number of rows
n = 100000

# Pickup locations: Either Airport or one of the hotels
hotels = ['Mallorca Sunrise', 'Palma Retreat', 'Balearic Bliss', 'Coastline Comfort', 'Mediterranean Marvel']
pickups = np.random.choice(['Airport'] + hotels, n)

# Dropoff locations
dropoffs = [np.random.choice(hotels) if pickup == 'Airport' else 'Airport' for pickup in pickups]

# Pickup times
base_time = datetime.strptime('08:00', '%H:%M')
pickup_times = [(base_time + timedelta(minutes=15*i)).time().strftime('%H:%M') for i in range(n)]

# Approx service times
service_times = np.random.randint(15, 110, n)

# Create a DataFrame
df = pd.DataFrame({
    'Pickup_Location': pickups,
    'Dropoff_Location': dropoffs,
    'Pickup_Time': pickup_times,
    'Aprox_Service_Time (mins)': service_times
})

# Save to CSV
df.to_csv('dummy_data.csv', index=False)


In [3]:
df

,Pickup_Location,Dropoff_Location,Pickup_Time,Aprox_Service_Time (mins)
0,Balearic Bliss,Airport,08:00,18
1,Coastline Comfort,Airport,08:15,95
2,Palma Retreat,Airport,08:30,99
3,Coastline Comfort,Airport,08:45,96
4,Coastline Comfort,Airport,09:00,61
...,...,...,...,...
99995,Mediterranean Marvel,Airport,22:45,90
99996,Coastline Comfort,Airport,23:00,23
99997,Balearic Bliss,Airport,23:15,58
99998,Airport,Balearic Bliss,23:30,74


In [4]:

locations = list(set(df['Pickup_Location'].unique().tolist() + df['Dropoff_Location'].unique().tolist()))
locations

['Mallorca Sunrise',
 'Mediterranean Marvel',
 'Coastline Comfort',
 'Palma Retreat',
 'Balearic Bliss',
 'Airport']

In [5]:

# TODO: Replace these distance calculations with your actual distances.
distances = {(loc1, loc2): np.random.randint(10, 100) if loc1 != loc2 else 0 for loc1 in locations for loc2 in locations}
distances

{('Mallorca Sunrise', 'Mallorca Sunrise'): 0,
 ('Mallorca Sunrise', 'Mediterranean Marvel'): 94,
 ('Mallorca Sunrise', 'Coastline Comfort'): 71,
 ('Mallorca Sunrise', 'Palma Retreat'): 53,
 ('Mallorca Sunrise', 'Balearic Bliss'): 55,
 ('Mallorca Sunrise', 'Airport'): 98,
 ('Mediterranean Marvel', 'Mallorca Sunrise'): 83,
 ('Mediterranean Marvel', 'Mediterranean Marvel'): 0,
 ('Mediterranean Marvel', 'Coastline Comfort'): 23,
 ('Mediterranean Marvel', 'Palma Retreat'): 22,
 ('Mediterranean Marvel', 'Balearic Bliss'): 30,
 ('Mediterranean Marvel', 'Airport'): 53,
 ('Coastline Comfort', 'Mallorca Sunrise'): 12,
 ('Coastline Comfort', 'Mediterranean Marvel'): 88,
 ('Coastline Comfort', 'Coastline Comfort'): 0,
 ('Coastline Comfort', 'Palma Retreat'): 15,
 ('Coastline Comfort', 'Balearic Bliss'): 13,
 ('Coastline Comfort', 'Airport'): 88,
 ('Palma Retreat', 'Mallorca Sunrise'): 81,
 ('Palma Retreat', 'Mediterranean Marvel'): 25,
 ('Palma Retreat', 'Coastline Comfort'): 73,
 ('Palma Retreat'

In [6]:

K = 50  # 50 drivers available at the start of the day

# Create the problem
# The goal is to minimize the objective (which will be the total distance).
prob = LpProblem("vehicle", LpMinimize)

# Indicator variable if location i is connected to location j in the tour
x = LpVariable.dicts('x', distances, 0, 1, LpBinary)

# Dummy vars to eliminate subtours
u = LpVariable.dicts('u', locations, 0, len(locations) - 1, LpInteger)

# Objective
cost = lpSum([x[(i, j)] * distances[(i, j)] for (i, j) in distances])
prob += cost

# Constraints
for loc in locations:
    # Each location should have only one inbound and one outbound connection
    prob += lpSum([x[(i, loc)] for i in locations if (i, loc) in x]) == 1
    prob += lpSum([x[(loc, i)] for i in locations if (loc, i) in x]) == 1

# Subtour elimination
N = len(locations)
for i in locations:
    for j in locations:
        if i != j and (i, j) in x:
            prob += u[i] - u[j] <= (N) * (1 - x[(i, j)]) - 1

# Solve
prob.solve()
print(LpStatus[prob.status])

# Extract the routes
non_zero_edges = [e for e in x if value(x[e]) != 0]

def get_next_site(parent):
    '''helper function to get the next edge'''
    edges = [e for e in non_zero_edges if e[0] == parent]
    for e in edges:
        non_zero_edges.remove(e)
    return edges

start_location = df['Pickup_Location'][0]  # Taking the first pickup location as the starting point
tours = get_next_site(start_location)
tours = [[e] for e in tours]

for t in tours:
    while t[-1][1] != start_location:
        t.append(get_next_site(t[-1][1])[-1])

for t in tours:
    print(' -> '.join([a for a, b in t] + [start_location]))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/nil/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/9d39623ccca541aca688fb90ff8ab0f9-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/9d39623ccca541aca688fb90ff8ab0f9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 47 COLUMNS
At line 324 RHS
At line 367 BOUNDS
At line 410 ENDATA
Problem MODEL has 42 rows, 42 columns and 162 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 30 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 30 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 30 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 30 strengthened rows, 0 substitutions
Cgl0004I processed model has 42 rows, 42 columns (42 in

In [ ]:
Not working with more than k>4...